# Intro
The following code can be used to generate labels for electronic devices like screw terminals, pin headers, etc.

In [ ]:
!pip install pillow

## Program Dependencies

In [1]:
from PIL import Image, ImageDraw, ImageFont
import os

pixels_per_inch = 300
inch_to_mm = 25.4

# Label: (background color, text color)
default_colors = {
    "GND": ("black", "white"),
    # Power supply labels
    "5V": ("red", "black"),
    "3V3": ("red", "black"),
    # Data labels
    "RX": ("pink", "black"),
    "TX": ("green", "black"),
    # Clock labels
    "CLK": ("yellow", "black"),
    "SCK": ("yellow", "black"),
 
}

def mm_to_pixels(mm):
    return int((mm / inch_to_mm) * pixels_per_inch)

def pixels_to_mm(pixels):
    return (pixels / pixels_per_inch) * inch_to_mm

def textsize(text, font):
    im = Image.new(mode="P", size=(0, 0))
    draw = ImageDraw.Draw(im)
    _, _, width, height = draw.textbbox((0, 0), text=text, font=font)
    return width, height

class Label:
    """Class to represent a single label.

    @param text: The text for the label.
    @param width: Width of the label in mm.
    @param height: Height of the label in mm.
    @param background_color: The background color of the label (optional).
    @param text_color: The color of the label (optional).
    """

    def __init__(self, text, width, height, background_color="white", text_color="black"):
        self._text = text
        self.width = width
        self.height = height
        self.background_color = background_color
        self.text_color = text_color

    @property
    def text(self):
        return self._text
    
    @text.setter
    def text(self, text):
        self._text = text
        # Set default colors based on the text
        self.background_color = default_colors.get(text, ("white", "black"))[0]
        self.text_color = default_colors.get(text, ("white", "black"))[1]

    def generate_label_image(self):
        """Generate an image for the terminal label."""
        width_pixels = mm_to_pixels(self.width)
        height_pixels = mm_to_pixels(self.height)

        # Create a blank image with the background color
        image = Image.new("RGB", (width_pixels, height_pixels), self.background_color)
        draw = ImageDraw.Draw(image)

        # Load a font
        try:
            if os.name == "posix": # macOS or Linux
                font_path = "/System/Library/Fonts/Supplemental/DIN Condensed Bold.ttf"
            elif os.name == "nt": # Windows
                font_path = "C:\\Windows\\Fonts\\bahnschrift.ttf"
                font = ImageFont.truetype(font_path, size=int(height_pixels * 0.75))
                font.set_variation_by_name("Condensed")
        except IOError:
            font = ImageFont.load_default()

        # Calculate text size and position
        text_width, text_height = textsize(self.text, font=font)
        text_x = (width_pixels - text_width) / 2
        text_y = (height_pixels - text_height) / 2

        # Draw the text on the image
        draw.text((text_x, text_y), self.text, fill=self.text_color, font=font)

        # Draw a border around the label
        border_width = 1
        draw.rectangle([0, 0, width_pixels - 1, height_pixels - 1], outline="black", width=border_width)

        return image

class LabelGrid:
    """Class to represent a screw label grid.

    @param number_of_columns: Number of columns in the grid.
    @param number_of_rows: Number of rows in the grid.
    @param start_pin: The pin number of the first label.
    @param length: Length of the label grid label section in mm.
    @param height: Height of the label grid label section in mm.

    """
    def __init__(self, number_of_columns, number_of_rows, start_pin, length, height):
        self.number_of_columns = number_of_columns
        self.number_of_rows = number_of_rows
        self.start_pin = start_pin
        self.length = length
        self.width = height
        self.terminal_label = [Label] * number_of_columns * number_of_rows

        for i in range(number_of_columns * number_of_rows):
            label = f"{start_pin + i}"
            self.terminal_label[i] = Label(label, length / number_of_columns, height / number_of_rows)

    def generate_block_image(self):
        """Generate an image for the terminal block."""
        total_width_pixels = mm_to_pixels(self.length)
        height_pixels = mm_to_pixels(self.width)

        # Create a blank image for the terminal block
        block_image = Image.new("RGB", (total_width_pixels, height_pixels), "black")
        
        # Calculate width of each terminal label
        terminal_width_pixels = total_width_pixels / self.number_of_columns
        
        for i, terminal in enumerate(self.terminal_label):
            terminal_image = terminal.generate_label_image()
            # Calculate position to paste the terminal label
            x_position = int(i * terminal_width_pixels) % total_width_pixels
            y_position = int(i / self.number_of_columns) * mm_to_pixels(terminal.height)
            block_image.paste(terminal_image, (x_position, y_position))

        return block_image


def print_grid(block_image, grid_rows, grid_columns, spacing_mm=5):
    """
    Create a grid of the block image for printing.

    :param block_image: The block image to replicate in the grid.
    :param grid_rows: Number of rows in the grid.
    :param grid_columns: Number of columns in the grid.
    :param spacing_mm: Spacing between images in millimeters.
    :return: A grid image.
    """
    # Convert spacing from mm to pixels
    spacing_pixels = mm_to_pixels(spacing_mm)

    # Get dimensions of the block image
    block_width, block_height = block_image.size

    # Calculate dimensions of the final grid image
    grid_width = grid_columns * block_width + (grid_columns - 1) * spacing_pixels
    grid_height = grid_rows * block_height + (grid_rows - 1) * spacing_pixels

    # Create a blank canvas for the grid
    grid_image = Image.new("RGB", (grid_width, grid_height), "white")

    # Paste block images onto the grid
    for row in range(grid_rows):
        for col in range(grid_columns):
            x = col * (block_width + spacing_pixels)
            y = row * (block_height + spacing_pixels)
            grid_image.paste(block_image, (x, y))

    return grid_image





## Program Input

In [2]:
terminal_block = LabelGrid(5, 1, 5, 19, 3.5)
terminal_block.terminal_label[0].text = "1"
terminal_block.terminal_label[1].text = "2"
terminal_block.terminal_label[2].text = "IN"
terminal_block.terminal_label[3].text = "RX"
terminal_block.terminal_label[3].background_color = "yellow"
terminal_block.terminal_label[4].text = "5V"
terminal_block.terminal_label[4].background_color = "red"
block_image = terminal_block.generate_block_image()
block_image.show()
block_image.save("terminal_block_1.png")

terminal_block = LabelGrid(5, 1, 9, 19, 3.5)
terminal_block.terminal_label[0].text = "GND"
terminal_block.terminal_label[0].background_color = "black"
terminal_block.terminal_label[0].text_color = "white"
terminal_block.terminal_label[1].text = "SWD"
terminal_block.terminal_label[1].background_color = "yellow"
terminal_block.terminal_label[2].text = "CLK"
terminal_block.terminal_label[2].background_color = "green"
terminal_block.terminal_label[3].text = "OUT"
terminal_block.terminal_label[4].text = "TX"
terminal_block.terminal_label[4].background_color = "green"
block_image = terminal_block.generate_block_image()
block_image.show()
block_image.save("terminal_block_2.png")

terminal_block = LabelGrid(2, 4, 1, 5, 10)
terminal_block.terminal_label[0].text = "TX"
terminal_block.terminal_label[1].text = "GND"
terminal_block.terminal_label[2].text = "RX"
terminal_block.terminal_label[3].text = "DIO"
terminal_block.terminal_label[4].text = "B0"
terminal_block.terminal_label[5].text = "3V3"
terminal_block.terminal_label[6].text = "RST"
terminal_block.terminal_label[7].text = "CLK"
block_image = terminal_block.generate_block_image()
grid_image = print_grid(block_image, grid_rows=3, grid_columns=4, spacing_mm=5)
grid_image.show()  # Display the grid
grid_image.save("grid_image.png")  # Save the grid image
